#### Prepared for Gabor's Data Analysis

### Data Analysis for Business, Economics, and Policy
by Gabor Bekes and  Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.

### Chapter 09
**CH09A Estimating gender and age differences in earnings**

using the cps-earnings dataset

version 1.0 2021-05-05

In [1]:
import os
import sys
import warnings

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import norm
from IPython.core.display import HTML
from stargazer.stargazer import Stargazer
import statsmodels.nonparametric.kernel_regression as loess

warnings.filterwarnings("ignore")


ModuleNotFoundError: No module named 'mizani'

In [ ]:
# Current script folder
current_path = os.getcwd()
dirname = current_path.split("da_case_studies")[0]

# location folders
data_in = dirname + "da_data_repo/cps-earnings/clean/"
data_out = dirname + "da_case_studies/ch09-gender-age-earnings/"
output = dirname + "da_case_studies/ch09-gender-age-earnings/output/"
func = dirname + "da_case_studies/ch00-tech-prep/"
sys.path.append(func)


In [ ]:
# Import the prewritten helper functions
from py_helper_functions import *


In [ ]:
data_all = pd.read_csv(data_in + "morg-2014-emp.csv")


In [ ]:
# SELECT OCCUPATION
# keep only two occupation types: Market research analysts and marketing specialists
# and Computer and Mathematical Occupations
data_all.loc[data_all["occ2012"] == 735, "sample"] = 1
data_all.loc[
    ((data_all["occ2012"] >= 1005) & (data_all["occ2012"] <= 1240)), "sample"
] = 2
data_all.loc[data_all["sample"].isna(), "sample"] = 0


In [ ]:
data_all = data_all.loc[
    (data_all["sample"] == 1) | (data_all["sample"] == 2), :
].reset_index(drop=True)


In [ ]:
data_all["sample"].value_counts()


In [ ]:
data_all["female"] = (data_all.sex == 2).astype(int)
data_all["w"] = data_all["earnwke"] / data_all["uhours"]
data_all["lnw"] = np.log(data_all["w"])
data_all["agesq"] = np.power(data_all["age"], 2)


In [ ]:
i = 1
data = data_all.loc[data_all["sample"] == i, :].reset_index(drop=True)
data.to_csv(data_out + "earnings_inference.csv", index=False)


In [ ]:
#####################
# DISTRIBUTION OF EARNINGS
#######################
data.loc[:, ["earnwke", "uhours", "w"]].describe()


In [ ]:
data.loc[data.w >= 1, ["earnwke", "uhours", "w"]].describe()


In [ ]:
data["female"].value_counts()


In [ ]:
data.groupby(["occ2012", "female"]).size()


In [ ]:
##############################
# linear regressions
##############################

# First, look at them one by one


In [ ]:
reg1 = smf.ols(formula="lnw~female", data=data).fit()
reg1.summary()


In [ ]:
reg2 = smf.ols(formula="lnw~female", data=data).fit(cov_type="HC1")
reg2.summary()


### Table 9.1 Wage and gender gap baseline regression

In [ ]:
stargazer = Stargazer([reg1, reg2])
stargazer.covariate_order(["female", "Intercept"])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer


In [ ]:
reg3 = smf.ols(formula="lnw~age", data=data).fit(cov_type="HC1")
reg3.summary()


In [ ]:
reg4 = smf.ols(formula="lnw~age+agesq", data=data).fit(cov_type="HC1")
reg4.summary()


In [ ]:
reg5 = smf.ols(formula="lnw~lspline(age,[30,40])", data=data).fit(cov_type="HC1")
reg5.summary()


In [ ]:
reg6 = loess.KernelReg(data["lnw"], data["age"], var_type="c", reg_type="lc")
# loess.lowess(data['lnw'],data["age"],frac=0.75,return_sorted=False)
# reg6 = localr(y=data["lnw"], x=data["age"],kernel=gaussian,frac=0.75)


In [ ]:
reg6


### Table 9.2 Wage and age – different specifications

In [ ]:
stargazer = Stargazer([reg3, reg4, reg5])
stargazer.covariate_order(
    [
        "age",
        "agesq",
        "lspline(age, [30, 40])[0]",
        "lspline(age, [30, 40])[1]",
        "lspline(age, [30, 40])[2]",
        "Intercept",
    ]
)
stargazer.rename_covariates(
    {
        "Intercept": "Constant",
        "agesq": "age squared",
        "lspline(age, [30, 40])[0]": "age spline <30",
        "lspline(age, [30, 40])[1]": "age spline 30–40",
        "lspline(age, [30, 40])[2]": "age spline 40<",
    }
)
stargazer


### Figure 9.3 Log hourly wage and age: regressions that capture nonlinearity

(a) Lowess regression and scatterplot

In [ ]:
##############################
# graphs
##############################
(
    ggplot(data, aes(x="age", y="lnw"))
    + geom_point(color=color[0])
    + geom_smooth(method="loess", color=color[1], se=False)
    + scale_x_continuous(expand=(0.01, 0.01), limits=(20, 65), breaks=seq(20, 65, by=5))
    + scale_y_continuous(
        expand=(0.01, 0.01), limits=(1.5, 4.5), breaks=seq(1.5, 4.5, by=0.50)
    )
    + labs(x="Age (years)", y="ln(earnings per hour)")
    + theme_bw()
)


In [ ]:
z = reg4.get_prediction().conf_int()


In [ ]:
res = reg4.get_prediction().summary_frame()
data["lnwpred_ageq"] = res["mean"]
data["lnwpred_ageqCIUP"] = [x[0] for x in z]
data["lnwpred_ageqCILO"] = [x[1] for x in z]


In [ ]:
z = reg5.get_prediction().conf_int()


In [ ]:
res = reg5.get_prediction().summary_frame()
data["lnwpred_agesp"] = res["mean"]
data["lnwpred_agespCIUP"] = [x[0] for x in z]
data["lnwpred_agespCILO"] = [x[1] for x in z]


In [ ]:
data["lnwpred_agel"] = reg6.fit()[0]


(b) Lowess, piecewise linear spline, and quadratic

In [ ]:
(
    ggplot(data, aes(x="age"))
    + geom_line(
        data,
        aes(y="lnwpred_agel"),
        color=color[0],
        linetype="solid",
        size=1.2,
        show_legend=True,
    )
    + geom_line(
        data,
        aes(y="lnwpred_ageq"),
        color=color[2],
        linetype="dotted",
        size=1.2,
        show_legend=True,
    )
    + geom_line(
        data,
        aes(y="lnwpred_agesp"),
        color=color[1],
        linetype="dashed",
        size=1.2,
        show_legend=True,
    )
    + scale_x_continuous(expand=(0.01, 0.01), limits=(20, 65), breaks=seq(20, 66, by=5))
    + scale_y_continuous(
        expand=(0.01, 0.01), limits=(2.4, 3.6), breaks=seq(2.4, 3.6, by=0.20)
    )
    + labs(x="Age (years)", y="ln(earnings per hour)")
    + theme_bw()
    + theme(
        legend_position=(45, 2.6),
        legend_direction="horizontal",
        legend_text=element_text(size=3),
        legend_key_width=0.8,
        legend_key_height=0.2,
    )
    + guides(linetype=guide_legend(override_aes=dict(size=0.6)))
    + scale_color_discrete(name=" ", values=["red", "cyan", "green"])
)


### Figure 9.4 Average log earnings and age: regressions with CI

In [ ]:
(
    ggplot(data, aes(x="age"))
    + geom_line(aes(y="lnwpred_agel"), color=color[0], linetype="solid", size=1.2)
    + geom_line(aes(y="lnwpred_ageq"), color=color[2], linetype="dotted", size=1.2)
    + geom_line(aes(y="lnwpred_ageqCIUP"), color=color[2], linetype="dotted", size=0.6)
    + geom_line(aes(y="lnwpred_ageqCILO"), color=color[2], linetype="dotted", size=0.6)
    + geom_ribbon(aes(ymin="lnwpred_ageqCILO", ymax="lnwpred_ageqCIUP"), alpha=0.2)
    + geom_line(aes(y="lnwpred_agesp"), color=color[1], linetype="dashed", size=1.2)
    + geom_line(aes(y="lnwpred_agespCIUP"), color=color[1], linetype="dashed", size=0.6)
    + geom_line(aes(y="lnwpred_agespCILO"), color=color[1], linetype="dashed", size=0.6)
    + geom_ribbon(aes(ymin="lnwpred_agespCILO", ymax="lnwpred_agespCIUP"), alpha=0.2)
    + coord_cartesian(xlim=(20, 65), ylim=(2.6, 3.6))
    + scale_x_continuous(expand=(0.01, 0.01), limits=(20, 65), breaks=seq(20, 65, by=5))
    + scale_y_continuous(
        expand=(0.01, 0.01), limits=(2.4, 3.6), breaks=seq(2.4, 3.6, by=0.20)
    )
    + labs(x="Age (years)", y="ln(earnings per hour)")
    + scale_color_manual(name="", values=(color[1], color[2], color[3]))
    + scale_linetype_manual(name="", values=("solid", "dashed", "dotted"))
    + theme_bw()
    + theme(
        legend_position=(0.65, 0.1),
        legend_direction="horizontal",
        legend_text=element_text(size=4),
        legend_key_width=0.8,
        legend_key_height=0.2,
    )
    + guides(linetype=guide_legend(override_aes=dict(size=0.6)))
)


### Figure 9.2 Log hourly wage and age: regression line, confidence interval, prediction interval.

In [ ]:
##########################################
# CI and PI for the linear model
##########################################
reg7 = smf.ols(
    formula="lnw~age",
    data=data.loc[
        data["sample"] == 1,
    ],
).fit()


(a) Confidence interval

In [ ]:
pred_confidence = data.join(
    pd.DataFrame(reg7.get_prediction().conf_int(), columns=["lwr", "upr"])
).join(reg7.get_prediction().summary_frame()["mean"].rename("fit"))


In [ ]:
(
    ggplot(
        pred_confidence.loc[lambda x: (x["lnw"] < 4.4) & (x["lnw"] > 2)],
        aes(x="age", y="lnw"),
    )
    + geom_point(color=color[0], size=1, alpha=0.8, show_legend=False, na_rm=True)
    + geom_smooth(method="lm", colour=color[1], se=False, size=0.8, linetype="solid")
    + geom_line(
        pred_confidence,
        aes(x="age", y="lwr"),
        size=0.5,
        linetype="dashed",
        colour=color[1],
    )
    + geom_line(
        pred_confidence,
        aes(x="age", y="upr"),
        size=0.5,
        linetype="dashed",
        colour=color[1],
    )
    + coord_cartesian(xlim=(20, 65), ylim=(1.5, 4.5))
    + scale_x_continuous(expand=(0.01, 0.01), limits=(20, 65), breaks=seq(20, 65, by=5))
    + scale_y_continuous(
        expand=(0.01, 0.01), limits=(1.5, 4.5), breaks=seq(1.5, 4.5, by=0.50)
    )
    + labs(x="Age (years)", y="ln(earnings per hour)")
    + scale_linetype_manual(
        name="",
        values=(1, 1, 2),
        labels=("Lowess", "Confidence interval (95%)", "Confidence interval (95%)"),
    )
    + theme_bw()
)


(b) Prediction interval

In [ ]:
pred_interval = data
pred_interval[["lwr", "upr"]] = (
    reg7.get_prediction().summary_frame().loc[:, ["obs_ci_lower", "obs_ci_upper"]]
)
pred_interval = pred_interval.join(
    reg7.get_prediction().summary_frame()["mean"].rename("fit")
)


In [ ]:
(
    ggplot(
        pred_interval.loc[lambda x: (x["lnw"] < 4.4) & (x["lnw"] > 2)],
        aes(x="age", y="lnw"),
    )
    + geom_point(color=color[0], size=1, alpha=0.8, show_legend=False, na_rm=True)
    + geom_smooth(method="lm", colour=color[1], se=False, size=0.8, linetype="solid")
    + geom_line(
        pred_interval, aes(y="lwr"), size=0.5, linetype="dashed", colour=color[1]
    )
    + geom_line(
        pred_interval, aes(y="upr"), size=0.5, linetype="dashed", colour=color[1]
    )
    + coord_cartesian(xlim=(20, 65), ylim=(1.5, 4.5))
    + scale_x_continuous(expand=(0.01, 0.01), limits=(20, 65), breaks=seq(20, 65, by=5))
    + scale_y_continuous(
        expand=(0.01, 0.01), limits=(1.5, 4.5), breaks=seq(1.5, 4.5, by=0.50)
    )
    + labs(x="Age (years)", y="ln(earnings per hour)")
    + theme_bw()
)


### Figure 9.1 Bootstrap distribution of the average female–male wage difference among market analysts

In [ ]:
data = pd.read_csv(data_out + "earnings_inference.csv")


In [ ]:
def bs_linreg(x, y, size=1, seed=200999):
    """Perform pairs bootstrap for linear regression."""
    # Set up array of indices to sample from
    inds = np.arange(len(x))

    # Initialize samples
    bs_slope_reps = np.empty(size)
    bs_intercept_reps = np.empty(size)
    np.random.seed(seed)
    # Take samples
    for i in range(size):
        bs_inds = np.random.choice(inds, len(inds), replace=True)
        bs_x, bs_y = sm.add_constant(x[bs_inds]), y[bs_inds]
        bs_slope_reps[i], bs_intercept_reps[i] = (
            sm.regression.linear_model.OLS(bs_y, bs_x).fit().params
        )

    return bs_slope_reps, bs_intercept_reps


In [ ]:
results = bs_linreg(data.lnw.values, data.female.values, size=1000)

b_earnings_female = pd.DataFrame(results).T

b_earnings_female.columns = ["_b_intercept", "_b_female"]


In [ ]:
(
    ggplot(b_earnings_female, aes(x="_b_female"))
    + geom_histogram(
        aes(y="stat(count)/sum(stat(count))"),
        binwidth=0.025,
        center=0.0125,
        closed="left",
        color="white",
        fill=color[0],
        size=0.2,
        alpha=0.8,
        show_legend=False,
        na_rm=True,
    )
    + geom_segment(
        aes(
            x=b_earnings_female["_b_female"].mean(),
            y=0,
            xend=b_earnings_female["_b_female"].mean(),
            yend=0.2,
        ),
        colour=color[1],
        size=1,
    )
    + annotate("text", x=-0.07, y=0.18, label="mean", size=12.5)
    + coord_cartesian(xlim=(-0.3, 0.15), ylim=(0, 0.2))
    + labs(x="Slope coefficients from bootstrap samples", y="Percent")
    + scale_y_continuous(expand=(0.0, 0.0), limits=(0, 0.2), labels=percent_format())
    + theme_bw()
)
